In [1]:
import numpy as np # library that supports large and multi-dimensional arrays and matrices
import pandas as pd # library for data manipulation and analysis
import scipy.io # to read data from and write data to a variety of file formats
from scipy import sparse # 2D sparse matrix package
from sklearn.decomposition import PCA # linear dimensionality reduction
from sklearn.linear_model import Ridge # linear least squares with L2 regularization
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from scipy.stats import zscore
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import math
from matplotlib.pyplot import *

In [2]:
class Reservoir(object):
    """
    Build a reservoir and evaluate internal states
    
    Parameters:
        n_internal_units = processing units in the reservoir
        spectral_radius = largest eigenvalue of the reservoir matrix of connection weights
        leak = amount of leakage in the reservoir state update (optional)
        connectivity = percentage of nonzero connection weights (unused in circle reservoir)
        input_scaling = scaling of the input connection weights
        noise_level = deviation of the Gaussian noise injected in the state update
        circle = generate determinisitc reservoir with circle topology
    """
    
    def __init__(self, n_internal_units=100, spectral_radius=0.99, leak=None,
                 connectivity=0.3, input_scaling=0.2, noise_level=0.01, circle=False):
        
        # Initialize attributes
        self._n_internal_units = n_internal_units
        self._input_scaling = input_scaling
        self._noise_level = noise_level
        self._leak = leak

        # Input weights depend on input size: they are set when data is provided
        self._input_weights = None

        # Generate internal weights
        if circle:
            self._internal_weights = self._initialize_internal_weights_Circ(
                    n_internal_units,
                    spectral_radius)
        else:
            self._internal_weights = self._initialize_internal_weights(
                n_internal_units,
                connectivity,
                spectral_radius)


    def _initialize_internal_weights_Circ(self, n_internal_units, spectral_radius):
        
        internal_weights = np.zeros((n_internal_units, n_internal_units))
        internal_weights[0,-1] = spectral_radius
        for i in range(n_internal_units-1):
            internal_weights[i+1,i] = spectral_radius
                
        return internal_weights
    
    
    def _initialize_internal_weights(self, n_internal_units,
                                     connectivity, spectral_radius):

        # Generate sparse, uniformly distributed weights.
        internal_weights = sparse.rand(n_internal_units,
                                       n_internal_units,
                                       density=connectivity).todense()

        # Ensure that the nonzero values are uniformly distributed in [-0.5, 0.5]
        internal_weights[np.where(internal_weights > 0)] -= 0.5
        
        # Adjust the spectral radius.
        E, _ = np.linalg.eig(internal_weights)
        e_max = np.max(np.abs(E))
        internal_weights /= np.abs(e_max)/spectral_radius       

        return internal_weights


    def _compute_state_matrix(self, X, n_drop=0):
        N, T, _ = X.shape
        previous_state = np.zeros((N, self._n_internal_units), dtype=float)

        # Storage
        state_matrix = np.empty((N, T - n_drop, self._n_internal_units), dtype=float)
        for t in range(T):
            current_input = X[:, t, :]

            # Calculate state
            state_before_tanh = self._internal_weights.dot(previous_state.T) + self._input_weights.dot(current_input.T)

            # Add noise
            state_before_tanh += np.random.rand(self._n_internal_units, N)*self._noise_level

            # Apply nonlinearity and leakage (optional)
            if self._leak is None:
                previous_state = np.tanh(state_before_tanh).T
            else:
                previous_state = (1.0 - self._leak)*previous_state + np.tanh(state_before_tanh).T

            # Store everything after the dropout period
            if (t > n_drop - 1):
                state_matrix[:, t - n_drop, :] = previous_state

        return state_matrix


    def get_states(self, X, n_drop=0, bidir=True):
        N, T, V = X.shape
        if self._input_weights is None:
            self._input_weights = (2.0*np.random.binomial(1, 0.5 , [self._n_internal_units, V]) - 1.0)*self._input_scaling

        # compute sequence of reservoir states
        states = self._compute_state_matrix(X, n_drop)
    
        # reservoir states on time reversed input
        if bidir is True:
            X_r = X[:, ::-1, :]
            states_r = self._compute_state_matrix(X_r, n_drop)
            states = np.concatenate((states, states_r), axis=2)

        return states
    
    def getReservoirEmbedding(self, X,pca, ridge_embedding,  n_drop=0, bidir=True, test = False):

        res_states = self.get_states(X, n_drop=0, bidir=True)

        N_samples = res_states.shape[0]
        res_states = res_states.reshape(-1, res_states.shape[2])                    
        # ..transform..
        if test:
            red_states = pca.transform(res_states)
        else:
            red_states = pca.fit_transform(res_states)          
        # ..and put back in tensor form
        red_states = red_states.reshape(N_samples,-1,red_states.shape[1])
        print("red_states:" + str(red_states.shape))

        coeff_tr = []
        biases_tr = []   
        
        for i in range(X.shape[0]):
            ridge_embedding.fit(red_states[i, 0:-1, :], red_states[i, 1:, :])
            coeff_tr.append(ridge_embedding.coef_.ravel())
            biases_tr.append(ridge_embedding.intercept_.ravel())
        #print(np.array(coeff_tr).shape,np.array(biases_tr).shape)
        input_repr = np.concatenate((np.vstack(coeff_tr), np.vstack(biases_tr)), axis=1)
        return input_repr

In [4]:
def targetify(s):
    if s == 'BENIGN':
        return 0
    else:
        return 1

In [5]:
def eqArray(a,b):
    return np.where(a == b, 1, 0)

In [14]:
df1 = pd.read_csv('oversampled_CICIDS2017_Wed.csv')
df2 = df1.groupby([' Timestamp'])[' Flow Duration'].count()
df2 = pd.DataFrame(df2).reset_index()
df2.columns=[' Timestamp','BiFlowsCount']
df = df1.merge(df2, left_on=' Timestamp', right_on=' Timestamp')
df = df.sort_values(' Timestamp')
num_features = 20 # should be 10, 15, or 20
features_ = ['BiFlowsCount', ' Protocol', ' ACK Flag Count', 'Idle Mean', ' Bwd Packet Length Mean', ' Flow IAT Max', ' Destination Port', 
             ' Avg Bwd Segment Size', ' Packet Length Std', ' Idle Max', ' Average Packet Size', ' Packet Length Mean', ' Fwd IAT Max', ' Source Port',
            ' min_seg_size_forward', ' Bwd Packet Length Std', ' Min Packet Length', ' URG Flag Count', 'Bwd Packet Length Max', ' Fwd IAT Std']
features = features_[0:num_features]
df

C:\Users\kdagi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,target,BiFlowsCount
533819,52770,192.168.10.3-192.168.10.15-53-58098-17,192.168.10.15,58098,192.168.10.3,53,17,5/7/2017 10:00,181186,4,...,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,BENIGN,0,1554
533949,65256,192.168.10.1-192.168.10.3-53-60930-17,192.168.10.3,60930,192.168.10.1,53,17,5/7/2017 10:00,59693,1,...,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,BENIGN,0,1554
533948,600244,192.168.10.3-192.168.10.9-53-57312-17,192.168.10.9,57312,192.168.10.3,53,17,5/7/2017 10:00,280632,2,...,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,BENIGN,0,1554
533947,50849,172.16.0.1-192.168.10.50-54178-80-6,172.16.0.1,54178,192.168.10.50,80,6,5/7/2017 10:00,118013760,2,...,0.000000e+00,87.0,87.0,1.180000e+08,0.000000e+00,118000000.0,118000000.0,DoS slowloris,1,1554
533946,52813,172.16.0.1-192.168.10.50-58374-80-6,172.16.0.1,58374,192.168.10.50,80,6,5/7/2017 10:00,3002965,3,...,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,DoS slowloris,1,1554
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602730,34148,192.168.10.14-104.88.37.207-50860-443-6,104.88.37.207,443,192.168.10.14,50860,6,5/7/2017 9:59,49,2,...,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,BENIGN,0,1175
602729,40899,192.168.10.3-192.168.10.5-123-123-17,192.168.10.5,123,192.168.10.3,123,17,5/7/2017 9:59,164,2,...,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,BENIGN,0,1175
602728,50836,172.16.0.1-192.168.10.50-58272-80-6,172.16.0.1,58272,192.168.10.50,80,6,5/7/2017 9:59,102589615,14,...,4.535367e+06,6414542.0,565.0,1.920000e+07,1.880000e+07,51300000.0,5772658.0,DoS slowloris,1,1175
602744,34158,192.168.10.9-74.125.22.157-15619-443-6,192.168.10.9,15619,74.125.22.157,443,6,5/7/2017 9:59,5141382,5,...,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,BENIGN,0,1175


In [15]:
del df['target']
del df['Unnamed: 0']
df.columns

Index(['Flow ID', ' Source IP', ' Source Port', ' Destination IP',
       ' Destination Port', ' Protocol', ' Timestamp', ' Flow Duration',
       ' Total Fwd Packets', ' Total Backward Packets',
       'Total Length of Fwd Packets', ' Total Length of Bwd Packets',
       ' Fwd Packet Length Max', ' Fwd Packet Length Min',
       ' Fwd Packet Length Mean', ' Fwd Packet Length Std',
       'Bwd Packet Length Max', ' Bwd Packet Length Min',
       ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s',
       ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max',
       ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std',
       ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean',
       ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags',
       ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags',
       ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s',
       ' Bwd Packets/s', ' Min Packet Length', ' Max Pa

In [17]:
fraction = 0.5
print(str(num_features) + " features")
print("fraction:" + str(fraction))
data = df.sample(frac=fraction, replace=True, random_state=1)

# get X and y. Normalize X and make it into 3D shape for reservoir
num_col = data.shape[1]
num_row = data.shape[0]

X_data = data[features]
X_data[features] = X_data[features].apply(pd.to_numeric, errors='coerce', axis=1)
#norm_scaler = preprocessing.StandardScaler()
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(X_data.values)
X = np.nan_to_num(x_scaled)
if len(X.shape) < 3:
    X = np.atleast_3d(X)
y = data[' Label'].apply(targetify)
print("Finished loading X and y......")

# split into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

print("X_train shape:" + str(X_train.shape), "y_train shape:" + str(y_train.shape))
print("X_test shape:" + str(X_test.shape), "y_test shape:" + str(y_test.shape))

pca = PCA() #n_components gives number of components to keep for linear dimensionality reduction
ridge_embedding = Ridge(alpha=10, fit_intercept=True)
readout = Ridge(alpha=5)

20 features
fraction:0.5


C:\Users\kdagi\Anaconda3\lib\site-packages\pandas\core\frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


Finished loading X and y......
X_train shape:(320000, 20, 1) y_train shape:(320000,)
X_test shape:(80000, 20, 1) y_test shape:(80000,)


In [73]:
n=30 #number of internal units
print(str(n) + " internal units")

#run through reservoir
res = Reservoir(n_internal_units=n, spectral_radius=0.9, leak=0.2,
     connectivity=0.25, input_scaling=0.3, noise_level=0.01, circle=False)
input_repr = res.getReservoirEmbedding(np.array(X_train), pca, ridge_embedding,  n_drop=0, bidir=False, test = False)
print("Finished loading training reservoir embedding......")
input_repr_te = res.getReservoirEmbedding(np.array(X_test), pca, ridge_embedding,  n_drop=0, bidir=False, test = True)
print("Finished loading testing reservoir embedding......")

#fit output
readout.fit(input_repr, y_train)
pred_class = readout.predict(input_repr_te)
#predictions = [int(round(x)) for x in pred_class]
true_class = list(y_test)

#analysis
compdf = pd.DataFrame({'pred_class':pred_class, 'true_class':true_class})
compdf = compdf.sort_values('pred_class', ascending=False)
print(str(compdf.head(10)))
#compdf.to_csv(str(dataset.split('_')[0]) + '_' + str(fraction) + '_' + str(num_features) + '_' + str(n) + '.csv')
#accuracy = np.sum(list(map(eqArray, predictions, true_class))) / len(true_class)
#f1 = f1_score(true_class, predictions)
#auc = roc_auc_score(true_class, predictions)

#print("# of nonzero:" + str(np.count_nonzero(predictions)))
#print("accuracy is " + str(accuracy))
#print("f1 is " + str(f1))
#print("auc is " + str(auc))
#print("*******************************************************************")

30 internal units
red_states:(320000, 20, 60)
Finished loading training reservoir embedding......
red_states:(80000, 20, 60)
Finished loading testing reservoir embedding......
       pred_class  true_class
56898    1.359128           1
60946    1.348427           1
49747    1.296080           1
43117    1.295769           1
40929    1.292817           1
9850     1.291667           1
4570     1.282292           1
35094    1.272470           1
51012    1.267673           1
35840    1.267554           1


In [74]:
def myRound(x, r):
    if x>r/float(1000):
        return 1
    else:
        return 0

In [75]:
predictions = list(compdf['pred_class'].apply(myRound, r=225))
true_class = list(compdf['true_class'])
accuracy = np.sum(list(map(eqArray, predictions, true_class))) / len(true_class)
accuracy

0.97115

In [76]:
from sklearn.metrics import confusion_matrix

In [77]:
def myRound(x, r):
    if x>r/float(1000):
        return 1
    else:
        return 0

In [78]:
confm = confusion_matrix(true_class, predictions)
confm

array([[37866,  2095],
       [  213, 39826]], dtype=int64)

In [79]:
tn, fp, fn, tp = confm.ravel()
(tn + tp)/(tn+tp+fn+fp)

0.97115

In [80]:
print("False alarm rate is")
fp/(tn+fp)

False alarm rate is


0.052426115462576015

In [81]:
print("Precision is")
Precision = tp/(tp+fp)
Precision

Precision is


0.9500250471124257

In [82]:
print("Recall is")
Recall = tp/(tp+fn)
Recall

Recall is


0.9946801868178526

In [83]:
print("F1 is")
2*Precision*Recall/(Precision+Recall)

F1 is


0.9718399219131284